In [1]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re

In [2]:
def mse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean() if reduction == "mean" else out 

In [24]:

p = '/data1/jessica/data/toyota/ckpts_final/'

experiments = os.listdir(p)
res = {}
res_ckpt = {}
for elem in experiments:
    path = p + elem + "/lightning_logs/" 
    vs = os.listdir(path)
    filt = []
    for elem1 in vs: 
        if 'version' in elem1:
            filt.append(elem1)
    versions =[elem.split("_")[-1]for elem in filt]
    versions = sorted(versions)
    version = f"version_{versions[-1]}"
    if elem == 'ckpts_final_comma_distance_clip' or elem == 'ckpts_final_comma_multitask_vit': version = "version_0"
    print(version, elem)

    checkpoint_path = path + version + "/checkpoints/"
    if "checkpoints" not in os.listdir(path + version): continue
    files = os.listdir(checkpoint_path)

    task = []
    ckpt = []
    files = sorted(files)
    for filename in files: 
        if filename.endswith(".csv"):
            df = pd.read_csv(checkpoint_path + filename)
            df.columns = ['preds', 'targets']
            m = (df['targets'] > 50).astype(bool) | (df['targets'] == 0).astype(bool) 
            
            loss3 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
            task.append(round(loss3.item(), 2))
        if filename.endswith(".ckpt"):
            ckpt.append(checkpoint_path + '/' + filename)
    res[elem] = task
    res_ckpt[elem] = ckpt


version_0 ckpts_final_comma_distance_resnet
version_0 ckpts_final_comma_multitask_none
version_6 ckpts_final_nuscenes_angle_vit
version_3 ckpts_final_nuscenes_multitask_none
version_5 ckpts_final_nuscenes_multitask_resnet
version_0 ckpts_final_comma_multitask_resnet
version_0 ckpts_final_comma_angle_resnet
version_8 ckpts_final_nuscenes_multitask_vit
version_5 ckpts_final_nuscenes_angle_resnet
version_7 ckpts_final_nuscenes_distance_vit
version_5 ckpts_final_nuscenes_multitask_clip
version_6 ckpts_final_nuscenes_angle_none
version_0 ckpts_final_comma_distance_none
version_5 ckpts_final_nuscenes_angle_clip
version_3 ckpts_final_nuscenes_distance_clip
version_0 ckpts_final_comma_multitask_vit
version_1 ckpts_final_comma_multitask_clip
version_0 ckpts_final_comma_distance_clip
version_7 ckpts_final_nuscenes_distance_none
version_0 ckpts_final_comma_angle_none
version_10 ckpts_final_comma_angle_vit
version_3 ckpts_final_nuscenes_distance_resnet
version_6 ckpts_final_comma_angle_clip
versio

In [25]:
os.listdir("/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_clip/lightning_logs")

['version_0']

In [26]:
res

{'ckpts_final_comma_distance_resnet': [4.67],
 'ckpts_final_comma_multitask_none': [4.58, 3.37],
 'ckpts_final_nuscenes_angle_vit': [20.48],
 'ckpts_final_nuscenes_multitask_none': [16.87, 7.45],
 'ckpts_final_nuscenes_multitask_resnet': [],
 'ckpts_final_comma_multitask_resnet': [2.83, 4.99],
 'ckpts_final_comma_angle_resnet': [3.16],
 'ckpts_final_nuscenes_multitask_vit': [],
 'ckpts_final_nuscenes_angle_resnet': [],
 'ckpts_final_nuscenes_multitask_clip': [20.1, 10.74],
 'ckpts_final_nuscenes_angle_none': [18.01],
 'ckpts_final_comma_distance_none': [3.39],
 'ckpts_final_nuscenes_angle_clip': [16.03],
 'ckpts_final_nuscenes_distance_clip': [],
 'ckpts_final_comma_multitask_vit': [2.96, 4.92],
 'ckpts_final_comma_multitask_clip': [],
 'ckpts_final_comma_distance_clip': [5.55],
 'ckpts_final_nuscenes_distance_none': [7.71],
 'ckpts_final_comma_angle_none': [5.16],
 'ckpts_final_comma_angle_vit': [],
 'ckpts_final_nuscenes_distance_resnet': [],
 'ckpts_final_comma_angle_clip': [2.12],


In [27]:
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    data = splitted[2]
    task = splitted[3]
    backbone = splitted[4]
    rows.append({"Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": round(res[elem][0], 2) if len(res[elem]) == 1 else res[elem]})



In [28]:
rows

[{'Task': 'distance',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 4.67},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'comma',
  't-RMSE': [4.58, 3.37]},
 {'Task': 'angle', 'Backbone': 'vit', 'Dataset': 'nuscenes', 't-RMSE': 20.48},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'nuscenes',
  't-RMSE': [16.87, 7.45]},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'nuscenes',
  't-RMSE': []},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': [2.83, 4.99]},
 {'Task': 'angle', 'Backbone': 'resnet', 'Dataset': 'comma', 't-RMSE': 3.16},
 {'Task': 'multitask', 'Backbone': 'vit', 'Dataset': 'nuscenes', 't-RMSE': []},
 {'Task': 'angle', 'Backbone': 'resnet', 'Dataset': 'nuscenes', 't-RMSE': []},
 {'Task': 'multitask',
  'Backbone': 'clip',
  'Dataset': 'nuscenes',
  't-RMSE': [20.1, 10.74]},
 {'Task': 'angle', 'Backbone': 'none', 'Dataset': 'nuscenes', 't-RMSE': 18.01},
 {'Task': 'distance', 'Backbone': 'none', 

In [29]:
df = pd.DataFrame(rows)
df

,Task,Backbone,Dataset,t-RMSE
0,distance,resnet,comma,4.67
1,multitask,none,comma,"[4.58, 3.37]"
2,angle,vit,nuscenes,20.48
3,multitask,none,nuscenes,"[16.87, 7.45]"
4,multitask,resnet,nuscenes,[]
5,multitask,resnet,comma,"[2.83, 4.99]"
6,angle,resnet,comma,3.16
7,multitask,vit,nuscenes,[]
8,angle,resnet,nuscenes,[]
9,multitask,clip,nuscenes,"[20.1, 10.74]"


In [30]:


# The list of dictionaries
data = rows

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = ['t-RMSE ' + col for col in df_pivot.columns]

# Reset the index
df_pivot = df_pivot.reset_index()
df_pivot['Feat. Size'] = [512, 643, 512, 768, 512, 643, 512, 768]
df_pivot = df_pivot[["Dataset", "Backbone", 'Feat. Size', 't-RMSE angle', 't-RMSE distance', 't-RMSE multitask']].round(2)
# Display the pivoted DataFrame
print(df_pivot.to_latex(index=False))


\begin{tabular}{llrlll}
\toprule
 Dataset & Backbone &  Feat. Size & t-RMSE angle & t-RMSE distance & t-RMSE multitask \\
\midrule
   comma &     clip &         512 &         2.12 &            5.55 &               [] \\
   comma &     none &         643 &         5.16 &            3.39 &     [4.58, 3.37] \\
   comma &   resnet &         512 &         3.16 &            4.67 &     [2.83, 4.99] \\
   comma &      vit &         768 &           [] &            5.02 &     [2.96, 4.92] \\
nuscenes &     clip &         512 &        16.03 &              [] &    [20.1, 10.74] \\
nuscenes &     none &         643 &        18.01 &            7.71 &    [16.87, 7.45] \\
nuscenes &   resnet &         512 &           [] &              [] &               [] \\
nuscenes &      vit &         768 &        20.48 &             NaN &               [] \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2415353/2574246896.py:18: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex(index=False))


In [31]:
df_pivot

,Dataset,Backbone,Feat. Size,t-RMSE angle,t-RMSE distance,t-RMSE multitask
0,comma,clip,512,2.12,5.55,[]
1,comma,none,643,5.16,3.39,"[4.58, 3.37]"
2,comma,resnet,512,3.16,4.67,"[2.83, 4.99]"
3,comma,vit,768,[],5.02,"[2.96, 4.92]"
4,nuscenes,clip,512,16.03,[],"[20.1, 10.74]"
5,nuscenes,none,643,18.01,7.71,"[16.87, 7.45]"
6,nuscenes,resnet,512,[],[],[]
7,nuscenes,vit,768,20.48,NaN,[]
